In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Setup Selenium WebDriver
driver = webdriver.Chrome()

url = 'https://www.highfieldqualifications.com/products/qualifications/'
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

# Find the div using the provided XPath and extract all URLs
div = driver.find_element(By.XPATH, '/html/body/app-root/app-base-layout/div/div[2]/main/app-landing-page/div[2]/div[3]')
links = div.find_elements(By.TAG_NAME, 'a')

# Print the URLs
urls = [link.get_attribute('href') for link in links]
print(urls)
print(len(urls))
# Close the driver
driver.quit()


['https://www.highfieldqualifications.com/products/qualifications/allergens', 'https://www.highfieldqualifications.com/products/qualifications/assessment', 'https://www.highfieldqualifications.com/products/qualifications/auditing-inspection', 'https://www.highfieldqualifications.com/products/qualifications/fundamental-british-values', 'https://www.highfieldqualifications.com/products/qualifications/business-administration', 'https://www.highfieldqualifications.com/products/qualifications/business-improvement-lean', 'https://www.highfieldqualifications.com/products/qualifications/children-young-people-workforce', 'https://www.highfieldqualifications.com/products/qualifications/cleaning', 'https://www.highfieldqualifications.com/products/qualifications/conflict-management', 'https://www.highfieldqualifications.com/products/qualifications/construction', 'https://www.highfieldqualifications.com/products/qualifications/coshh', 'https://www.highfieldqualifications.com/products/qualifications

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor


def scrape_page(url):
    """Function to scrape a page and extract links."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    driver.implicitly_wait(10)

    extracted_links = []
    try:
        div = driver.find_element(By.XPATH, '/html/body/app-root/app-base-layout/div/div[2]/main/app-sub-category/app-products/div/div/div[2]/div[2]')
        links = div.find_elements(By.TAG_NAME, 'a')
        extracted_links = [link.get_attribute('href') for link in links if link.get_attribute('href')]
    except Exception as e:
        print(f"Error processing {url}: {e}")

    driver.quit()
    return extracted_links

# Use ThreadPoolExecutor for parallel execution
all_links = []
with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
    results = executor.map(scrape_page, urls)

# Flatten the list
for result in results:
    all_links.extend(result)

# Print or use the collected links
print(all_links)


Error processing https://www.highfieldqualifications.com/products/qualifications/quality-assurance: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app-root/app-base-layout/div/div[2]/main/app-sub-category/app-products/div/div/div[2]/div[2]"}
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF71EF6C6A5+28789]
	(No symbol) [0x00007FF71EED5B20]
	(No symbol) [0x00007FF71ED68F9A]
	(No symbol) [0x00007FF71EDBF346]
	(No symbol) [0x00007FF71EDBF57C]
	(No symbol) [0x00007FF71EE12B17]
	(No symbol) [0x00007FF71EDE736F]
	(No symbol) [0x00007FF71EE0F7E3]
	(No symbol) [0x00007FF71EDE7103]
	(No symbol) [0x00007FF71EDAFFC0]
	(No symbol) [0x00007FF71EDB1273]
	GetHandleVerifier [0x00007FF71F2B1AED+3458237]
	GetHandleVerifier [0x00007FF71F2C829C+3550316]
	GetHandleVerifier [0x00007FF71F

In [1]:
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementClickInterceptedException
import time
import os

def download_qualification_spec(url):
    """Process a single URL to download the qualification specification"""
    # Set up Chrome options with custom download directory
    chrome_options = Options()
    prefs = {
        "download.default_directory": r"C:\Users\amith\Kenpath\OFQUAL\highfield_pdfs",
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True  # Automatically download PDFs instead of opening them
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument("--headless=new")  # Use new headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")  # Set a larger window size
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    
    driver = None
    try:
        # Initialize WebDriver
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        
        # Wait for the page to load completely
        time.sleep(3)
        
        # Click the "Qualification Documents" tab using JavaScript
        qualification_tab = driver.find_element(By.XPATH, "//li[contains(text(), 'Qualification Documents')]") 
        driver.execute_script("arguments[0].click();", qualification_tab)

        # Wait for the Qualification Documents section to appear
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//app-qualification-documents")))
                
        # Find the download link
        try:
            # Try to find and get the href attribute
            download_link_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//strong[contains(text(),'Qualification Specification')]/ancestor::div[contains(@class,'row')]//a[contains(@class,'btn')]"))
            )
            
            # Get the href value
            download_url = download_link_element.get_attribute('href')
            
            if download_url:
                # Direct navigation to the download URL
                print(f"Navigating directly to download URL: {download_url}")
                driver.get(download_url)
                time.sleep(2)  # Wait for download to start
            else:
                # If can't get href, try JavaScript click as fallback
                driver.execute_script("arguments[0].click();", download_link_element)
                time.sleep(5)
                print(f"Used JavaScript click for {url}")
        
        except Exception as e:
            print(f"Download link error: {e}")
            # Last resort approach
            try:
                # Try to find any download button for qualification specification
                elements = driver.find_elements(By.XPATH, "//a[contains(@class,'btn')]")
                for element in elements:
                    if "download" in element.get_attribute('innerHTML').lower() and "qualification specification" in driver.find_element(By.XPATH, "ancestor::div[contains(@class,'row')]").text.lower():
                        driver.execute_script("arguments[0].click();", element)
                        time.sleep(5)
                        print(f"Found and clicked using element search for {url}")
                        break
            except Exception as e2:
                print(f"All attempts failed for {url}: {e2}")
            
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
    finally:
        # Close the browser
        if driver:
            driver.quit()


# Read URLs from file
with open("highfield_links.txt", "r") as file:
    urls = [line.strip() for line in file if line.strip()]

print(f"Processing {len(urls)} URLs")

# Use ThreadPoolExecutor to process URLs in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(download_qualification_spec, urls)

print("All downloads completed")



Processing 433 URLs
Navigating directly to download URL: https://centres.highfieldqualifications.com//Assets/DownloadArea/QualificationDownloads/[10042024_0922]_L2_Healthy_Food_Special_Diets_Specification_v1.2_2024_.pdf
Navigating directly to download URL: https://centres.highfieldqualifications.com//Assets/DownloadArea/QualificationDownloads/[15062023_1216]_5_Qual_specification_v1.4_-_L3_Award_in_Undertaking_EPA_-_final_version_(May_2023).pdf
Navigating directly to download URL: https://centres.highfieldqualifications.com//Assets/DownloadArea/QualificationDownloads/[02052024_1442]_Qual_specification_-_L3_Assessor_v1.4_April_2024.pdf
Navigating directly to download URL: https://centres.highfieldqualifications.com//Assets/DownloadArea/QualificationDownloads/[02052024_0917]_L3_Allergens_Qual_Spec_welsh.pdf
Navigating directly to download URL: https://centres.highfieldqualifications.com//Assets/DownloadArea/QualificationDownloads/[02052024_1442]_Qual_specification_-_L3_Assessor_v1.4_April